In [271]:
# import sys
# sys.path.append("..")

In [1]:
import gurobipy
import pandas as pd
from IORFA import *
import numpy as np
import time
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.linear_model import LinearRegression

In [2]:
train_ratio = 0.5
val_ratio = 0.25
test_ratio = 0.25
seed = 42

In [3]:

## Create artificial data set with
n = 500
x1 = np.random.normal(loc = 0,scale=1, size=n)
x2 = np.random.normal(loc=0, scale=1, size=n)
x3 = np.random.normal(loc=0, scale=1, size=n)
x4 = np.random.normal(loc=0, scale=1, size=n)
x5 = np.random.normal(loc =0,scale=1, size=n)
x6 = np.random.normal(loc=0, scale=1, size=n)
x7 = np.random.normal(loc=0, scale=1, size=n)
x8 = np.random.normal(loc=0, scale=1, size=n)

simulated_data = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 
                                'x5': x5, 'x6': x6, 'x7': x7, 'x8': x8})

x = simulated_data[[f'x{i}' for i in range(1, 9)]]

orig_cols = simulated_data.columns

# Create an instance of the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Assuming you have a feature matrix X
# Apply the min-max scaling to the data
x = scaler.fit_transform(x)

y = 0.2*(x[:, 0] > 0.2)*(x[:, 5] < 0.5) + 0.3*x[:, 3]

y_bar = y

y = y_bar

x = np.array(x)
y = np.array(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=seed)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, 
                                                test_size=test_ratio/(test_ratio+val_ratio), random_state=seed)

In [4]:
x_train = pd.DataFrame(x_train, columns = orig_cols)
x_test = pd.DataFrame(x_test, columns = orig_cols)

In [5]:
T = DecisionTreeRegressor(max_depth=2)
T.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=2)

In [6]:
# Extract the rules from the decision tree
tree_rules = export_text(T, feature_names=list(orig_cols))

print("Tree Rules:")
print(tree_rules)

Tree Rules:
|--- x6 <= 0.50
|   |--- x4 <= 0.51
|   |   |--- value: [0.31]
|   |--- x4 >  0.51
|   |   |--- value: [0.39]
|--- x6 >  0.50
|   |--- x4 <= 0.56
|   |   |--- value: [0.12]
|   |--- x4 >  0.56
|   |   |--- value: [0.21]



In [7]:
import re

def parse_tree_rules(tree_rules_str):
    lines = tree_rules_str.split("\n")
    rules = []
    conditions = []

    for line in lines:
        if '---' in line:
            # Determine the level of the current condition by counting the leading '|'
            level = line.count("|", 0, line.index("---"))
            # Extract variable, operator, and value
            match = re.search(r"(x\d+)\s*([<>=]+)\s*([\d\.]+)", line)
            if match:
                var, op, value = match.groups()

                print(var, op, value)

                # Adjust the size of conditions list to match the current level
                conditions = conditions[:level]
                print()
                conditions.append(f"({var} {op} {value})")

        elif 'value:' in line:
            # Extract value
            value = re.findall(r"\[([\d\.]+)\]", line)[0]
            # Combine conditions to a single string and append to rules
            rule = " & ".join(conditions)
            rules.append((rule, float(value)))

    return conditions

In [8]:
parse_tree_rules(tree_rules)

x6 <= 0.50

x4 <= 0.51

x4 > 0.51

x6 > 0.50

x4 <= 0.56

x4 > 0.56



['(x6 <= 0.50)', '(x6 > 0.50)', '(x4 > 0.56)']

In [29]:
from sklearn.tree import _tree

def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    print(samples_count)
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = ""
        
        for p in path[:-1]:
            if rule != "":
                rule += " & "
            rule += str(p)

        rules += [rule]
        
    return rules


In [41]:
X = x_train.copy()

T = DecisionTreeRegressor(max_depth=3)
T.fit(x_train, y_train)
tree = T
rules,paths ,nodes_thresholds= get_rules(tree, list(orig_cols), None)

In [48]:
tree.tree_.feature[1]

3

In [35]:
rules

['(x6 <= 0.5) & (x4 > 0.513) & (x4 <= 0.795)',
 '(x6 > 0.5) & (x4 <= 0.561) & (x4 > 0.35)',
 '(x6 <= 0.5) & (x4 <= 0.513) & (x4 > 0.344)',
 '(x6 > 0.5) & (x4 > 0.561) & (x4 <= 0.711)',
 '(x6 > 0.5) & (x4 > 0.561) & (x4 > 0.711)',
 '(x6 <= 0.5) & (x4 <= 0.513) & (x4 <= 0.344)',
 '(x6 > 0.5) & (x4 <= 0.561) & (x4 <= 0.35)',
 '(x6 <= 0.5) & (x4 > 0.513) & (x4 > 0.795)']

In [34]:
nodes_thresholds= {0: 0.49958091974258423,
 1: 0.5131429731845856,
 2: 0.34435226023197174,
 5: 0.7948507368564606,
 8: 0.5608195662498474,
 9: 0.3499424010515213,
 12: 0.7114875018596649}

{0: 0.49958091974258423,
 1: 0.5131429731845856,
 2: 0.34435226023197174,
 5: 0.7948507368564606,
 8: 0.5608195662498474,
 9: 0.3499424010515213,
 12: 0.7114875018596649}

In [36]:
def get_rules_from_dict(nodes_thresholds, feature_names):
    rules = {}
    for node, threshold in nodes_thresholds.items():
        if node != _tree.TREE_UNDEFINED:
            feature_name = feature_names[node]
            rule = f"{feature_name} <= {threshold}"  # create a rule
            rules[node] = rule
    return rules


In [15]:
for i, rule in enumerate(rules):
    x_test[f'rule_{i}'] = x_test.eval(rule).astype(int)

NameError: name 'rules' is not defined

In [22]:
x_train

,x1,x2,x3,x4,x5,x6,x7,x8
0,0.565995,0.150262,0.914487,0.621817,0.236517,0.275295,0.176273,0.693809
1,0.533263,0.760463,0.510611,0.565015,0.513558,0.916188,0.516458,0.707496
2,0.611347,0.609587,0.281186,0.567339,0.490805,0.625635,0.499041,0.449888
3,0.697311,0.506747,0.489502,0.501055,0.690571,0.673597,0.476525,0.486249
4,0.770253,0.509938,0.331303,0.604782,0.451043,0.301199,0.651899,0.609349
...,...,...,...,...,...,...,...,...
245,0.571175,0.793486,0.572033,0.167986,0.594599,0.464439,0.479892,0.864835
246,0.419999,0.523910,0.525964,0.486322,0.620032,0.712436,0.336747,0.613825
247,0.628963,0.192685,0.674890,0.129966,0.440042,0.554789,0.118806,0.117355
248,0.660782,0.739503,0.430310,0.424036,0.505794,0.000000,0.105403,0.580457


In [17]:
# def linear_regression_pipeline(X_train, X_test, y_train, y_test):

#     model = LinearRegression()
#     model.fit(X_train, y_train)

#     return round(model.score(X_train, y_train), 3), round(model.score(X_test, y_test), 3), model.coef_

In [18]:
# train_R2, test_R2 = linear_regression_pipeline(x_train, x_test, y_train, y_test)

In [290]:
# train_R2, test_R2

Objective function

In [294]:
from sklearn.metrics import mean_squared_error
# Algo 8.1 LocalSearch

def InitialSteps(X, y, max_depth, alpha):

    X = X.copy()

    T = DecisionTreeRegressor(max_depth=max_depth)
    T.fit(x_train, y_train)

    rules = get_rules(T, list(orig_cols), None)
    for i, rule in enumerate(rules):
        X[f'rule_{i}'] = X.eval(rule).astype(int)
    
    rulefit = LinearRegression()
    rulefit.fit(X, y) # rulefit is LinReg because we added rules already :D
    SSE = mean_squared_error(rulefit.predict(X), y)

    ####### Needed to define Loss(T, X, y) ######
    is_leaf = np.logical_and(T.tree_.children_left == -1, T.tree_.children_right == -1)
    d = np.count_nonzero(~is_leaf) # recall complexity d is |T| = number of branching nodes (not 2^max_depth)
    ######

    n = y.shape[0]
    L_init = SSE*n + alpha*d

    print(f"SSE contributes {SSE*n*100/L_init}% to the initial loss")
    print(f"Complexity contributes {alpha*d*100/L_init}%")

    return L_init

In [299]:
InitialSteps(x_train, y_train, 3, alpha = 0.00001)

SSE contributes 1.2360291662818856e-23% to the initial loss
Complexity contributes 100.0%


7.000000000000001e-05

In [319]:
def nodes(T):

    lc = list(T.tree_.children_left)
    rc = list(T.tree_.children_right)

    nodes = [0] + lc + rc # nodes are root + left children + right_children
    nodes = list(set(nodes)) # but some children can be duplicates
    nodes = np.setdiff1d(nodes, np.array(-1)) # also -1 is not a node but indicates a node doesn't exist

    return nodes

In [324]:
def shuffle(arr):
    # need a fn for this because np.random.shuffle is not inplace
    arr_copy = arr.copy()
    np.random.shuffle(arr_copy)
    return arr_copy 

In [328]:
def find_leaf_nodes(T, t):
    # List to store the leaf nodes
    leaf_nodes = []

    # Recursive helper function
    def explore(node):
        # If this is a leaf node, add it to the list
        if T.tree_.children_left[node] == T.tree_.children_right[node] == -1:
            leaf_nodes.append(node)
        
        else:
            # If this is a branch node, explore its children
            if T.tree_.children_left[node] != -1:
                explore(T.tree_.children_left[node])
            if T.tree_.children_right[node] != -1:
                explore(T.tree_.children_right[node])

    # Start exploring from node t
    explore(t)

    return leaf_nodes

In [346]:
def construct_I(T, X, nodes_in_T_t):

    belongs_to = T.apply(X) # leaf node obs belong to

    I = []

    for i in range(X.shape[0]):

        if belongs_to[i] in nodes_in_T_t:

            I.append(i) 
    
    return I

In [371]:
T.tree_.children_left

array([ 1,  2, -1, -1,  5, -1, -1], dtype=int64)

In [ ]:
class Tree_Opt:

    def __init__(self, X, y, min_leaf_size) -> None:
        self.X = X
        self.y = y
        self.min_leaf_size = min_leaf_size

    def OptimizeNodeParallel(self, id, X_I, y_I):

        left_child, right_child = self.find_children(id)
        
        if left_child != -1 or right_child != -1: # not leaf
            
            T_para, error_para = self.BestParallelSplit(id, left_child,
                                                        right_child, 
                                                        X_I, 
                                                        y_I)
            
        else:


    def BestParallelSplit(self, id,
                          left_child, 
                          right_child, 
                          X_I, 
                          y_I):
        
        n, p = X_I.shape
        error_best = np.inf
        tree_mod = T.copy()

        for j in range(p):

            values = X_I[:, j]
            values = sorted(values)

            for i in range(0, n):

                b = (1/2)*(values[i] + values[i+1])

                # For the jth feature, get all rows where the feature is greater than b
                higher_than_b = X_I[:, j] > b

                # And where it's lower than or equal to b
                lower_or_equal_to_b = X_I[:, j] <= b
                X_I_higher = X_I[higher_than_b]

                X_I_lower = X_I[lower_or_equal_to_b]

                if len(X_I_higher) > self.min_leaf_size and len(X_I_lower) > self.min_leaf_size:

                    tree_mod.tree_.feature[id] = j
                    tree_mod.tree_.threshold[id] = b
                    rules = get_rules(tree_mod, list(orig_cols), None)
                    for i, rule in enumerate(rules):
                        self.X[f'rule_{i}'] = self.X.eval(rule).astype(int)

                    rulefit = LinearRegression()
                    rulefit.fit(self.X, self.y) # rulefit is LinReg because we added rules already :D
                    SSE = mean_squared_error(rulefit.predict(self.X), self.y)

                    if SSE < error_best:
                        error_best = SSE
                        best_tree = tree_mod







                # Calculate loss and update error if current error is lower than error_best





            


    

In [372]:
def find_children(self, id):

    left_child_cand = [i for i in self.T.tree_.children_left if i > id]
    right_child_cand  = [i for i in self.T.tree_.children_right if i > id]

    left_children = list(set(left_child_cand))
    right_children = list(set(right_child_cand))

    if len(left_children) == 0:
        left_children = [-1]
    
    if len(right_children) == 0:
        right_children = [-1]
    
    return min(left_children), min(right_children)

In [349]:
def LocalSearch(T, X, y, max_depth):
    
    # Extracting initial loss
    error_prev = InitialSteps(X, y, max_depth, alpha = 0.00001)
    
    X = np.array(X)
    y = np.array(y)
    
    for t in shuffle(nodes(T)):

        nodes_in_T_t = find_leaf_nodes(T, t)

        I = construct_I(T, X, nodes_in_T_t)

        T_t = OptimizeNodeParallel(T_t, X[I], y[I])

        

        # Replace tth node in T with T_t


